# NB-05 - Workqueue Story (marts-only)

## Purpose
Generate a decision-ready workqueue snapshot and memo from marts-only sources.

## Data Requirements
- **DS3: required**
- **DS0: optional** (only used if explicitly enabled)

## Parameters
- Top N: **25**
- Sort: `at_risk_amt DESC`

## Outputs
- `workqueue_memo_latest_week.md` (ASCII-only)
- `nb05_workqueue_top25.png`

## How to Run
1. Open this notebook
2. Restart Kernel
3. Run All
4. Confirm outputs exist and are updated


In [1]:
import os
from datetime import datetime
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt


In [2]:
# Parameters
QUEUE_CAPACITY = 250
TOP_N = 25


In [3]:
# Load marts only (DS3 required, DS0 optional)
BQ_PROJECT_ID = os.getenv("BQ_PROJECT_ID") or os.getenv("GOOGLE_CLOUD_PROJECT")
BQ_DATASET_ID = os.getenv("BQ_DATASET_ID")

if not BQ_PROJECT_ID or not BQ_DATASET_ID:
    raise RuntimeError("BigQuery not configured. Set BQ_PROJECT_ID and BQ_DATASET_ID.")

import pandas_gbq

# DS3 required (query top N only)
DS3_TABLE = "mart_workqueue_claims"

required_cols = [
    "desynpuf_id",
    "clm_id",
    "at_risk_amt",
    "denied_potential_allowed_proxy_amt",
    "payer_allowed_amt",
    "observed_paid_amt",
    "payer_yield_gap_amt",
    "aging_days",
    "p_denial",
    "top_denial_group",
    "top_next_best_action",
    "top_hcpcs",
    "top_denial_prcsg",
]

select_cols = ", ".join(required_cols)
query_ds3 = f"""
SELECT {select_cols}
FROM `{BQ_PROJECT_ID}.{BQ_DATASET_ID}.{DS3_TABLE}`
ORDER BY at_risk_amt DESC
LIMIT {TOP_N}
"""

ds3 = pandas_gbq.read_gbq(query_ds3, project_id=BQ_PROJECT_ID)

# DS0 optional (receipt only)
DS0_TABLE = "mart_exec_overview_latest_week"
try:
    query_ds0 = f"SELECT week_start, mix_stability_flag FROM `{BQ_PROJECT_ID}.{BQ_DATASET_ID}.{DS0_TABLE}` LIMIT 1"
    ds0 = pandas_gbq.read_gbq(query_ds0, project_id=BQ_PROJECT_ID)
except Exception:
    ds0 = pd.DataFrame()


C:\Users\Allen\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\auth\_default.py:108: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Downloading:   0%|          |

Downloading: 100%|██████████|

Downloading:   0%|          |

Downloading: 100%|██████████|

In [4]:
# --- Guardrails ---
REQUIRED_COLS = [
    "desynpuf_id",
    "clm_id",
    "at_risk_amt",
    "denied_potential_allowed_proxy_amt",
    "payer_allowed_amt",
    "observed_paid_amt",
    "payer_yield_gap_amt",
    "aging_days",
    "p_denial",
    "top_denial_group",
    "top_next_best_action",
    "top_hcpcs",
    "top_denial_prcsg",
]

assert ds3 is not None, "DF is None ? DS3 query did not return data."
missing = [c for c in REQUIRED_COLS if c not in ds3.columns]
assert not missing, f"Missing required columns: {missing}"

assert len(ds3) <= TOP_N, f"Expected <= {TOP_N} rows, got {len(ds3)}"

# Ensure sort order
if len(ds3) > 1:
    assert (ds3["at_risk_amt"].fillna(0).values[:-1] >= ds3["at_risk_amt"].fillna(0).values[1:]).all(),         "Expected df sorted by at_risk_amt DESC"


In [5]:
# Total at-risk sum for share calculation
query_total = (
    "SELECT SUM(at_risk_amt) AS total_at_risk_amt "
    f"FROM `{BQ_PROJECT_ID}.{BQ_DATASET_ID}.{DS3_TABLE}`"
)

total_df = pandas_gbq.read_gbq(query_total, project_id=BQ_PROJECT_ID)
if total_df.empty or total_df.iloc[0]["total_at_risk_amt"] is None:
    total_at_risk = 0.0
else:
    total_at_risk = float(total_df.iloc[0]["total_at_risk_amt"])

# Top25 sum from current df
if "at_risk_amt" in ds3.columns:
    top25_sum = float(ds3["at_risk_amt"].fillna(0).sum())
else:
    top25_sum = 0.0

share_pct = (top25_sum / total_at_risk * 100) if total_at_risk else 0.0


Downloading:   0%|          |

Downloading: 100%|██████████|

In [6]:
# Select ranking metric
ranking_metric_used = "at_risk_amt" if "at_risk_amt" in ds3.columns else None
if ranking_metric_used is None:
    raise ValueError("DS3 missing ranking metric at_risk_amt")

# Sort and select top N
queue_df = ds3.sort_values(ranking_metric_used, ascending=False).head(TOP_N).copy()


In [7]:
# Build display columns (omit missing, note omissions)
required_order = [
    "desynpuf_id",
    "clm_id",
    "at_risk_amt",
    "denied_potential_allowed_proxy_amt",
    "payer_allowed_amt",
    "observed_paid_amt",
    "payer_yield_gap_amt",
    "aging_days",
    "p_denial",
    "top_denial_group",
    "top_next_best_action",
    "top_hcpcs",
    "top_denial_prcsg",
]

present_cols = [c for c in required_order if c in queue_df.columns]
missing_cols = [c for c in required_order if c not in queue_df.columns]

# Explainability column from marts fields only
explain_parts = []
for col in ["top_denial_group", "top_next_best_action", "top_hcpcs", "top_denial_prcsg"]:
    if col in queue_df.columns:
        explain_parts.append(col)

if explain_parts:
    def build_explain(row):
        parts = []
        if "top_denial_group" in explain_parts and pd.notna(row.get("top_denial_group")):
            parts.append(str(row.get("top_denial_group")))
        if "top_next_best_action" in explain_parts and pd.notna(row.get("top_next_best_action")):
            parts.append(str(row.get("top_next_best_action")))
        if "top_hcpcs" in explain_parts and pd.notna(row.get("top_hcpcs")):
            parts.append("HCPCS " + str(row.get("top_hcpcs")))
        if "top_denial_prcsg" in explain_parts and pd.notna(row.get("top_denial_prcsg")):
            parts.append("PRCSG " + str(row.get("top_denial_prcsg")))
        return " | ".join(parts) if parts else ""

    queue_df["Explain"] = queue_df.apply(build_explain, axis=1)
    present_cols = present_cols + ["Explain"]
else:
    missing_cols.append("Explain")


In [8]:
# Format for display
fmt_df = queue_df[present_cols].copy()

money_cols = [
    "at_risk_amt",
    "denied_potential_allowed_proxy_amt",
    "payer_allowed_amt",
    "observed_paid_amt",
    "payer_yield_gap_amt",
]
rate_cols = ["p_denial"]

for col in money_cols:
    if col in fmt_df.columns:
        fmt_df[col] = fmt_df[col].apply(lambda v: f"${float(v):,.0f}" if pd.notna(v) else "")

for col in rate_cols:
    if col in fmt_df.columns:
        fmt_df[col] = fmt_df[col].apply(lambda v: f"{float(v)*100:,.1f}%" if pd.notna(v) else "")


In [9]:
# Render table to PNG (fallback to CSV if failure)
img_dir = Path("docs") / "images" if Path("docs").is_dir() else Path("..") / "docs" / "images"
img_dir.mkdir(parents=True, exist_ok=True)
img_path = img_dir / "nb05_workqueue_top25.png"

csv_path = Path("docs") / "workqueue_top25.csv" if Path("docs").is_dir() else Path("..") / "docs" / "workqueue_top25.csv"

png_written = False
try:
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.axis('off')
    table = ax.table(
        cellText=fmt_df.values.tolist(),
        colLabels=fmt_df.columns.tolist(),
        loc='center'
    )
    table.auto_set_font_size(False)
    table.set_fontsize(7)
    table.scale(1, 1.2)
    fig.savefig(img_path, dpi=150, bbox_inches='tight')
    plt.close(fig)
    png_written = True
    print(f"Wrote {img_path}")
except Exception as exc:
    png_written = False
    fmt_df.to_csv(csv_path, index=False)
    print(f"PNG render failed; wrote {csv_path}")


Wrote ..\docs\images\nb05_workqueue_top25.png


In [10]:
# Memo export (ASCII-only)
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

anchor_week = "N/A"
if not ds0.empty and "week_start" in ds0.columns:
    try:
        anchor_week = str(ds0["week_start"].iloc[0])
    except Exception:
        anchor_week = "N/A"

fields_omitted_note = ""
if missing_cols:
    fields_omitted_note = "- Fields omitted: " + ", ".join(missing_cols)

memo_lines = [
    "# Workqueue Memo - Prioritized Claims (Demo)",
    "",
    "## What this is",
    "This is a prioritization demo built from DS3 marts outputs. It is not a guarantee of recoveries.",
    "",
    "## Receipt",
    f"- Anchor week (if DS0 available): {anchor_week}",
    f"- Ranking basis: {ranking_metric_used}",
    f"- Top N shown: {TOP_N}",
    f"- Capacity assumption: {QUEUE_CAPACITY}/day",
    "- Source: mart_workqueue_claims (DS3)",
    f"- Generated on: {now}",
    "",
    "## Queue preview (Top N)",
    f"Top 25 at-risk: ${top25_sum:,.0f} ({share_pct:.1f}% of total ${total_at_risk:,.0f})",
]

if png_written:
    memo_lines.append("![Workqueue Top N](images/nb05_workqueue_top25.png)")
else:
    memo_lines.append("See workqueue_top25.csv")

memo_lines += [
    "",
    "## How to use this",
    "- Start with the highest at-risk exposure.",
    "- Use the explainability columns to route to the correct workflow (action/category).",
    "- If mix stability is CHECK SEGMENTS in NB-03, treat queue as provisional until drivers are validated.",
    "",
    "## Guardrails",
    "- Proxy values are directional prioritization only; not guaranteed recovery.",
    "- Paid vs Allowed differences are claim-file amounts used for directional triage, not adjudicated underpayment findings.",
]

if fields_omitted_note:
    memo_lines.append("")
    memo_lines.append(fields_omitted_note)

memo_lines = [line.encode("ascii", "ignore").decode("ascii") for line in memo_lines]

memo_path = Path("docs") / "workqueue_memo_latest_week.md" if Path("docs").is_dir() else Path("..") / "docs" / "workqueue_memo_latest_week.md"
with open(memo_path, "w", encoding="utf-8") as handle:
    handle.write("\n".join(memo_lines))

print(f"Wrote {memo_path}")


Wrote ..\docs\workqueue_memo_latest_week.md
